![](../img/b15_analysis.png)

<div style="page-break-after: always;"></div> 

# 데이터 정제 및 분석 응용

<img src="https://www.tomasbeuzen.com/python-programming-for-data-science/_static/logo.png" alt="python for data science" width="300" height="300">

1. 문자열 작업

2. 날짜 및 시간 작업

3. 계층적 인덱싱

4. 데이터프레임 시각화

5. 판다스 프로파일링

<div style="page-break-after: always;"></div> 

<h2>학습 목표<span class="tocSkip"></span></h2>

- `Series.str` 속성으로 판다스 문자열을 처리할 수 있다.
- 문자열 정제를 위하여 정규 표현식을 사용할 수 있다. 
- `Timestamp`, `Timedelta`, `Period`, `DateOffset`과 같은  
  `datetime` 객체를 용도에 맞게 사용할 수 있다. 
- `pd.Timestamp()`, `pd.Period()`, `pd.date_range()`, `pd.period_range()` 등을 써서  
  다양한 `datetime` 객체를 생성할 수 있다.  
- 문자열 인덱싱을 써서 `datetime` 인덱스를 구성할 수 있다. 
- `datetime` 자료로부터 시간 요소 추출, 시차 적용, 시간대 변경, 표본 추출 등과 같은  
  기본적 `datetime` 연산을 처리할 수 있다.  
- `.plot` 속성을 이용하거나, `pandas.plotting`이 제공하는 함수를 활용하여  
  기본적 차트 시각화를 작성할 수 있다. 
  
<div style="page-break-after: always;"></div>

## 1. 문자열 작업

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", 20)  # https://wikidocs.net/75844

- 텍스트 자료에 대한 작업은 데이터 과학에서 일상적이다.  
  판다스 `Series`와 `Index` 객체는 문자열 처리를 위한 다양한 메소드를 제공한다.  
  이에 대하여 공부해 보자. 

- 문자열 작업에 관한 공부를  
  1.1 항 `String` 자료형부터 시작하자. 
  
<div style="page-break-after: always;"></div> 

### 1.1 `String` 자료형

- 문자열 자료는 (판다스에서) `object` 자료형으로 표현된다.  
  - `object` 자료형은 자료형이 혼합되었거나 크기를 모르는 자료를 총괄하는 자료형이다. 
  - 문자열을 위한 전용 자료형을 제공하는 것이 더 바람직한데,  
    판다스는 `StringDtype`이란 문자열 전용 자료형을 제공한다.
  - 본 교재의 편역 시점을 기준으로 최신 판다스는 2021년 9월 12일에 배포된 1.3.3 판인데,  
    `StringDtype` 자료형은 1.0 판부터 도입되었다.   
  - 그럼에도 불구하고, (아직까지는 과거 버전에 대한 호환성때문에)  
    `object` 자료형이 여전히 문자열에 대한 기본 자료형으로 지정되어 있다. 
  - 판다스 개발팀은 `string` 자료형을 지속적으로 테스트하면서 개선하고 있는 것으로 보인다.  
    [`StringDtype`에 관한 판다스 측의 자료](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#text-data-types)를 참고하라. 

- 1.1 항에서 `String` 자료형을 공부했다.
- 1.2 항에서 문자열 메소드를 공부하자.
  
<div style="page-break-after: always;"></div> 

### 1.2 문자열 메소드

- 속도 향상과 사용성 강화를 위하여 넘파이나 판다스와 같은 라이브러리는  
  벡터화 연산을 아래 예제와 같이 적용하고 있다.

In [2]:
x = np.array([1, 2, 3, 4, 5])
x * 2

array([ 2,  4,  6,  8, 10])

- 그러나 문자열 배열에 대해서는 이런 벡터화 연산이 적용되지 않는다.  
  아래 예제에서 이를 확인할 수 있다:

In [3]:
x = np.array(['Tom', 'Mike', 'Tiffany', 'Joel', 'Varada'])
x.upper()  # 대문자로 변환하는 upper() 메소드를 넘파이 배열에 적용 시도하면 오류 발생함

AttributeError: 'numpy.ndarray' object has no attribute 'upper'

- 문자열 배열에 대해서는 벡터화 연산이 적용되지 않기 때문에,  
  아래와 같이 컴프리헨션을 써서  
  모든 문자열 객체에 대하여 반복적으로 연산을 적용해야 한다: 


In [4]:
[name.upper() for name in x]

['TOM', 'MIKE', 'TIFFANY', 'JOEL', 'VARADA']

- 그런데 만일 문자열 배열이 결측치를 포함하고 있다면,  
  이마저도 오류를 유발한다: 

In [5]:
x = np.array(['Tom', 'Mike', None, 'Tiffany', 'Joel', 'Varada'])
[name.upper() for name in x]   

AttributeError: 'NoneType' object has no attribute 'upper'

- 사실 위 코드에서 `None` 대신에 `np.nan`을 썼다면 오류를 피할 수 있었다. 

In [6]:
x = np.array(['Tom', 'Mike', np.nan, 'Tiffany', 'Joel', 'Varada'])
[name.upper() for name in x]                                        # `np.nan`을 'NAN'으로 출력

['TOM', 'MIKE', 'NAN', 'TIFFANY', 'JOEL', 'VARADA']

- 판다스 개발팀에서도 문자열 메소드에 대한  
  이와 같은 (벡터화 및 결측치) 문제에 대하여 언급하고 있다.  
  - **문자열 메소드**는 판다스 `Series`나 `Index` 객체의 **`.str` 속성을 통하여 접근**이 가능하다. 
  - `.upper()`, `.lower()`, `.split()` 등과 같은  
    내장 문자열 메소드의 거의 대부분에 대하여 위 방법이 적용 가능하다. 

In [7]:
s = pd.Series(x)   # 리스트 x를 판다스 시리즈로 변환 
s

0        Tom
1       Mike
2        nan
3    Tiffany
4       Joel
5     Varada
dtype: object

In [8]:
s.str.upper()      # `Series`/`Index`의 str 속성을 통하여 접근 가능함, 원본 불변

0        TOM
1       MIKE
2        NAN
3    TIFFANY
4       JOEL
5     VARADA
dtype: object

In [9]:
s.str.split("ff", expand=True)  # str.split()으로 단일 열 시리즈를 두 열 데이터프레임으로 변환

,0,1
0,Tom,None
1,Mike,None
2,nan,None
3,Ti,any
4,Joel,None
5,Varada,None


In [10]:
s.str.split('a')                # expand=False, 열을 확장하지 않고, 단일 열 내부에서 리스트 원소로 분할 

0          [Tom]
1         [Mike]
2         [n, n]
3     [Tiff, ny]
4         [Joel]
5    [V, r, d, ]
dtype: object

In [11]:
s.str.len()                     # `NaN`에 대해서는 len() 반환값이 `NaN`

0    3
1    4
2    3
3    7
4    4
5    6
dtype: int64

- (행 인덱스이든 열 레이블이든) `index` 객체에 대해서도 이 방법이 적용 가능하다. 
- 아래 예제에서는 행 인덱스 및 열 레이블 정제에 관한 시범을 보인다. 

In [12]:
df = pd.DataFrame(np.random.rand(5, 3),  # [0, 1) 범위에서 균등분포 추출
                  columns = ['Measured Feature', 'recorded feature', 'PredictedFeature'],
                  index = [f"ROW{_}" for _ in range(5)])
df

,Measured Feature,recorded feature,PredictedFeature
ROW0,0.279854,0.470911,0.411151
ROW1,0.991914,0.021193,0.809973
ROW2,0.595749,0.206254,0.420522
ROW3,0.459546,0.523272,0.543596
ROW4,0.717176,0.557691,0.193651


- 열 레이블에서 대소문자와 띄어쓰기 일관성이 부족함 점을 개선해 보자. 
- 행 인덱스에서 문자와 숫자가 붙어 있는 점을 개선해 보자. 

In [13]:
df.columns

Index(['Measured Feature', 'recorded feature', 'PredictedFeature'], dtype='object')

In [14]:
type(df.columns)  # 데이터프레임 columns 객체는 Index 클래스

pandas.core.indexes.base.Index

- 열 레이블을 수정해 보자: 
  1. 열 레이블에서 첫 글자는 모두 대문자로 통일하고, "feature" 및 "Feature" 부분을 모두 제거 
  2. "ROW"를 소문자로 변환하고, 행 번호와 문자열 사이에 `_`를 삽입

In [15]:
df.columns                                                                      # 열 레이블 확인

Index(['Measured Feature', 'recorded feature', 'PredictedFeature'], dtype='object')

In [16]:
df.columns.str.capitalize()                                                      # 둘째 열에 주목

Index(['Measured feature', 'Recorded feature', 'Predictedfeature'], dtype='object')

In [17]:
df.columns.str.capitalize().str.replace("feature", "")                           # 1~2 열에는 후미 공백이 포함되었음

Index(['Measured ', 'Recorded ', 'Predicted'], dtype='object')

In [18]:
df.columns.str.capitalize().str.replace("feature", "").str.strip()               # 후미 공백 제거

Index(['Measured', 'Recorded', 'Predicted'], dtype='object')

In [19]:
df.columns = df.columns.str.capitalize().str.replace("feature", "").str.strip()  # 원본에 대입

- 행 인덱스를 수정해 보자. 

In [20]:
df.index                                               # 열 레이블 확인

Index(['ROW0', 'ROW1', 'ROW2', 'ROW3', 'ROW4'], dtype='object')

In [21]:
df.index.str.lower()                                    # 소문자로 변환

Index(['row0', 'row1', 'row2', 'row3', 'row4'], dtype='object')

In [22]:
df.index.str.lower().str.replace("w", "w_")             # 문자와 숫자 사이에 '_' 추가

Index(['row_0', 'row_1', 'row_2', 'row_3', 'row_4'], dtype='object')

In [23]:
df.index = df.index.str.lower().str.replace("w", "w_")  # 원본에 대입

In [24]:
df                                                      # 정제를 마친 행 인덱스와 열 레이블

,Measured,Recorded,Predicted
row_0,0.279854,0.470911,0.411151
row_1,0.991914,0.021193,0.809973
row_2,0.595749,0.206254,0.420522
row_3,0.459546,0.523272,0.543596
row_4,0.717176,0.557691,0.193651


- 멋지게 완성되었네요!  
  판다스의 문자열 메소드는 엄청나게 다양하다.  
  판다스가 제공하는 모든 문자열 메소드를 정리하면 다음과 같다.  
  (슬라이드 쇼를 멈추고 원본 노트북을 확인하자.)

| Method                     | Description                                                                                                                       |
| -------------------------- | --------------------------------------------------------------------------------------------------------------------------------- |
| `Series.str.cat`           | Concatenate strings                                                                                                               |
| `Series.str.split`         | Split strings on delimiter                                                                                                        |
| `Series.str.rsplit`        | Split strings on delimiter working from the end of the string                                                                     |
| `Series.str.get`           | Index into each element (retrieve i-th element)                                                                                   |
| `Series.str.join`          | Join strings in each element of the Series with passed separator                                                                  |
| `Series.str.get_dummies`   | Split strings on the delimiter returning DataFrame of dummy variables                                                             |
| `Series.str.contains`      | Return boolean array if each string contains pattern/regex                                                                        |
| `Series.str.replace`       | Replace occurrences of pattern/regex/string with some other string or the return value of a callable given the occurrence         |
| `Series.str.repeat`        | Duplicate values (`s.str.repeat(3)` equivalent to `x * 3`)                                                                        |
| `Series.str.pad`           | "Add whitespace to left, right, or both sides of strings"                                                                         |
| `Series.str.center`        | Equivalent to `str.center`                                                                                                        |
| `Series.str.ljust`         | Equivalent to `str.ljust`                                                                                                         |
| `Series.str.rjust`         | Equivalent to `str.rjust`                                                                                                         |
| `Series.str.zfill`         | Equivalent to `str.zfill`                                                                                                         |
| `Series.str.wrap`          | Split long strings into lines with length less than a given width                                                                 |
| `Series.str.slice`         | Slice each string in the Series                                                                                                   |
| `Series.str.slice_replace` | Replace slice in each string with passed value                                                                                    |
| `Series.str.count`         | Count occurrences of pattern                                                                                                      |
| `Series.str.startswith`    | Equivalent to `str.startswith(pat)` for each element                                                                              |
| `Series.str.endswith`      | Equivalent to `str.endswith(pat)` for each element                                                                                |
| `Series.str.findall`       | Compute list of all occurrences of pattern/regex for each string                                                                  |
| `Series.str.match`         | "Call `re.match` on each element, returning matched groups as list"                                                               |
| `Series.str.extract`       | "Call `re.search` on each element, returning DataFrame with one row for each element and one column for each regex capture group" |
| `Series.str.extractall`    | "Call `re.findall` on each element, returning DataFrame with one row for each match and one column for each regex capture group"  |
| `Series.str.len`           | Compute string lengths                                                                                                            |
| `Series.str.strip`         | Equivalent to `str.strip`                                                                                                         |
| `Series.str.rstrip`        | Equivalent to `str.rstrip`                                                                                                        |
| `Series.str.lstrip`        | Equivalent to `str.lstrip`                                                                                                        |
| `Series.str.partition`     | Equivalent to `str.partition`                                                                                                     |
| `Series.str.rpartition`    | Equivalent to `str.rpartition`                                                                                                    |
| `Series.str.lower`         | Equivalent to `str.lower`                                                                                                         |
| `Series.str.casefold`      | Equivalent to `str.casefold`                                                                                                      |
| `Series.str.upper`         | Equivalent to `str.upper`                                                                                                         |
| `Series.str.find`          | Equivalent to `str.find`                                                                                                          |
| `Series.str.rfind`         | Equivalent to `str.rfind`                                                                                                         |
| `Series.str.index`         | Equivalent to `str.index`                                                                                                         |
| `Series.str.rindex`        | Equivalent to `str.rindex`                                                                                                        |
| `Series.str.capitalize`    | Equivalent to `str.capitalize`                                                                                                    |
| `Series.str.swapcase`      | Equivalent to `str.swapcase`                                                                                                      |
| `Series.str.normalize`     | Return Unicode normal form. Equivalent to `unicodedata.normalize`                                                                 |
| `Series.str.translate`     | Equivalent to `str.translate`                                                                                                     |
| `Series.str.isalnum`       | Equivalent to `str.isalnum`                                                                                                       |
| `Series.str.isalpha`       | Equivalent to `str.isalpha`                                                                                                       |
| `Series.str.isdigit`       | Equivalent to `str.isdigit`                                                                                                       |
| `Series.str.isspace`       | Equivalent to `str.isspace`                                                                                                       |
| `Series.str.islower`       | Equivalent to `str.islower`                                                                                                       |
| `Series.str.isupper`       | Equivalent to `str.isupper`                                                                                                       |
| `Series.str.istitle`       | Equivalent to `str.istitle`                                                                                                       |
| `Series.str.isnumeric`     | Equivalent to `str.isnumeric`                                                                                                     |
| `Series.str.isdecimal`     | Equivalent to `str.isdecimal`                                                                                                     |

- 문자열 치환을 위한 `df.replace()` 메소드는 알아 둘 만하다. 

In [25]:
df = pd.DataFrame({'col_1': ['replace me', 'b', 'c'],
                   'col_2': [1, 99999, 3]})
df

,col_1,col_2
0,replace me,1
1,b,99999
2,c,3


In [26]:
df.replace({'replace me': 'a',    # 문자열 치환
            99999: 2})            #   수치 치환
                                  # 사전 형태로 치환 내역 지정 

,col_1,col_2
0,a,1
1,b,2
2,c,3


- 1.2 항에서 문자열 메소드를 공부했다.
- 1.3 항에서 정규 표현식을 공부하자.
  
<div style="page-break-after: always;"></div> 

### 1.3 정규 표현식

- 정규 표현식(regular expression (regex))은 탐색 패턴을 정의하는 일련의 문자열이다. 
  - 복잡한 문자열 작업을 하려면 정규 표현식을 사용할 줄 알아야 한다. 
  - 파이썬만의 고유 문법이 아니라 문자열을 처리하는 모든 곳에서 사용한다. 
  - 줄여서 '정규식'으로 쓰기도 한다. 
  - [점프 투 파이썬의 설명](https://wikidocs.net/1642)을 참고하라. 

- 정규 표현식을 입력 인자로 받아들이는 판다스 문자열 함수가 많다.  
  - 아래는 정규 표현식을 입력 인자로 받아들이는 대표적인 판다스 문자열 함수의 예이다.
  - 아래 설명에서 `re`는 정규식을 처리하는 파이썬 패키지 이름이다. 

|메소드|설명|
|---|---|
|`match()`|탐색 대상 요소마다 re.match()를 호출하고, 논리값을 반환 
|`extract()`|탐색 대상 요소마다 re.match()를 호출하고, 부합하는 그룹을 문자열로 반환 
|`findall()`|탐색 대상 요소마다 re.findall()을 호출 
|`replace()`|부합하는 패턴을 다른 문자열로 치환 
|`contains()`|탐색 대상 요소마다 re.search()를 호출하고, 논리값을 반환 
|`count()`|부합하는 패턴의 개수 
|`split()`|str.split()과 같지만, 정규 표현식을 입력 인자로 허용 
|`rsplit()`|str.rsplit()과 같지만, 정규 표현식을 입력 인자로 허용 

- 예를 들어서, `Series` 내부에서 단일 자음으로 시작하고 끝나는 모든 단어를 쉽게 탐색할 수 있다: 

In [27]:
s = pd.Series(['Tom', 'Mike', None, 'Tiffany', 'Joel', 'Varada'])
s

0        Tom
1       Mike
2       None
3    Tiffany
4       Joel
5     Varada
dtype: object

In [28]:
s.str.findall(r'^[^AEIOU].*[^aeiou]$')  # (Tom, Tiffany, Joel)만 패턴에 부합

0        [Tom]
1           []
2         None
3    [Tiffany]
4       [Joel]
5           []
dtype: object

- 직전 코드에 적용된 정규식 요소는 다음과 같다:

|요소|설명|
|---|---|
|`^`|`^` 부분은 문자열의 시작을 지정한다.|
|`[^AEIOU]`|대괄호 부분은 한 글자에 해당하는 패턴을 지정한다. <br>대괄호 내부에서 `^`를 쓰면 '부정'이라는 의미인데, <br>첫 글자가 대문자 모음('A', 'E', 'I', 'O', 및 'U')이 아니라는 의미이다. |
|`.*`|`.` 부분은 모든 문자를 의미하며, `*` 부분은 "0회 이상"이라는 의미이다. <br>따라서 문자열의 시작과 끝을 제외한 중간에 <br>어떤 문자가 몇 개 등장하더라도 문제 삼지 않겠다는 의미이다. |
|`[^aeiou]`|대괄호 부분은 한 글자에 해당하는 패턴을 지정한다. <br>대괄호 내부에서 `^`를 쓰면 '부정'이라는 의미인데, <br>마지막 글자가 소문자 모음('a', 'e', 'i', 'o', 및 'u')이 아니라는 의미이다.|
|`$`| `$` 부분은 문자열의 끝을 지정한다. |

- 직전 코드에서 `r`이라고 문자열 상수 앞에 붙은 부분은  
  가공하지 않은 원래 상태의 '생 문자열 상수(raw string literal)'라는 의미이다. 
- 아래 예제에서 일반 문자열 상수와 생 문자열 상수의 차이를 확인하라
  - 일반 문자열 상수에서는 `'\n'`이 줄바꿈 문자에 해당하는 이스케이프 코드로 처리된다. 
  - 생 문자열 상수에서는 `'\n'`이 줄바꿈 문자에 해당하는 이스케이프 코드를 무시하고 그냥 문자열로 처리된다. 


In [29]:
str = 'Hi\nHello'  # 일반 문자열 상수
print(str)

Hi
Hello


In [30]:
str = r'Hi\nHello'  # 생 문자열 상수
print(str)

Hi\nHello


- 복잡한 텍스트를 정제해야 하는 상황이라면 정규식이 마법같은 역햘을 할 수 있다.  
  자전거 데이터 집합에서 한 가지 예제를 더 살펴 보자:

In [31]:
df = pd.read_csv('../data/cycling_data.csv', index_col=0)
df

,Name,Type,Time,Distance,Comments
Date,,,,,
"10 Sep 2019, 00:13:04",Afternoon Ride,Ride,2084,12.62,Rain
"10 Sep 2019, 13:52:18",Morning Ride,Ride,2531,13.03,rain
"11 Sep 2019, 00:23:50",Afternoon Ride,Ride,1863,12.52,Wet road but nice weather
"11 Sep 2019, 14:06:19",Morning Ride,Ride,2192,12.84,Stopped for photo of sunrise
"12 Sep 2019, 00:28:05",Afternoon Ride,Ride,1891,12.48,Tired by the end of the week
...,...,...,...,...,...
"4 Oct 2019, 01:08:08",Afternoon Ride,Ride,1870,12.63,"Very tired, riding into the wind"
"9 Oct 2019, 13:55:40",Morning Ride,Ride,2149,12.70,Really cold! But feeling good
"10 Oct 2019, 00:10:31",Afternoon Ride,Ride,1841,12.59,Feeling good after a holiday break!


- `Comments` 열에 "Rain" 또는 "rain"이 포함된 행을 다음과 같이 탐색할 수 있다:

In [32]:
df.loc[df['Comments'].str.contains(r"[Rr]ain")]

,Name,Type,Time,Distance,Comments
Date,,,,,
"10 Sep 2019, 00:13:04",Afternoon Ride,Ride,2084,12.62,Rain
"10 Sep 2019, 13:52:18",Morning Ride,Ride,2531,13.03,rain
"17 Sep 2019, 13:43:34",Morning Ride,Ride,2285,12.60,Raining
"18 Sep 2019, 13:49:53",Morning Ride,Ride,2903,14.57,Raining today
"26 Sep 2019, 00:13:33",Afternoon Ride,Ride,1860,12.52,raining


- 만약 위 탐색 결과에서 "Raining"이나 "raining"은 배제하고 싶다면 다음과 같이 할 수 있다:

In [33]:
df.loc[df['Comments'].str.contains(r"^[Rr]ain$")]

,Name,Type,Time,Distance,Comments
Date,,,,,
"10 Sep 2019, 00:13:04",Afternoon Ride,Ride,2084,12.62,Rain
"10 Sep 2019, 13:52:18",Morning Ride,Ride,2531,13.03,rain


- `df.str.split()` 메소드를 써서 한 열의 문자열을 두 열로 분리할 수 있다. 
  - 성과 이름이 공백으로 구분되어 `df.name` 열에 저장되어 있다고 가정하면,  
    `df['name'].str.split(' ', expand=True)` 명령으로 
    성과 이름을 별도의 열로 분리할 수도 있다. 

    

  - 심지어 (마침표, 쉼표 및 느낌표를 포함하는) 구두점을 기반으로  
    문자열을 분할하거나 새로운 열로 분리할 수도 있다:


In [34]:
df['Comments'].str.split(r"[.,!]", expand=True)

,0,1
Date,,
"10 Sep 2019, 00:13:04",Rain,None
"10 Sep 2019, 13:52:18",rain,None
"11 Sep 2019, 00:23:50",Wet road but nice weather,None
"11 Sep 2019, 14:06:19",Stopped for photo of sunrise,None
"12 Sep 2019, 00:28:05",Tired by the end of the week,None
...,...,...
"4 Oct 2019, 01:08:08",Very tired,riding into the wind
"9 Oct 2019, 13:55:40",Really cold,But feeling good
"10 Oct 2019, 00:10:31",Feeling good after a holiday break,


- 직전 예제의 열 레이블을 확인하고, 이를 수정해 보자. 

In [35]:
df['Comments'].str.split(r"[.,!]", expand=True).columns

RangeIndex(start=0, stop=2, step=1)

In [36]:
df['Comments'].str.split(r"[.,!]", expand=True).rename(
    columns={0: 'Comments_0', 1:'Comments_1'})

,Comments_0,Comments_1
Date,,
"10 Sep 2019, 00:13:04",Rain,None
"10 Sep 2019, 13:52:18",rain,None
"11 Sep 2019, 00:23:50",Wet road but nice weather,None
"11 Sep 2019, 14:06:19",Stopped for photo of sunrise,None
"12 Sep 2019, 00:28:05",Tired by the end of the week,None
...,...,...
"4 Oct 2019, 01:08:08",Very tired,riding into the wind
"9 Oct 2019, 13:55:40",Really cold,But feeling good
"10 Oct 2019, 00:10:31",Feeling good after a holiday break,


- 정규 표현식에 대한 모든 것을 지금 알아야 하는 것은 아니다. 
  - 향후에 문자열에 대한 복잡한 작업을 수행해야 할 상황이 되면,  
    '정규표현식을 써서 쉽게 해결할 수 있다고 하던데...'라는 생각이 들기를 바란다. 
  - 필요할 때, 검색하고 공부하면 된다. 

- 1.3 항에서 정규 표현식을 공부했다.
- 이제 날짜/시간 조작 방법을 공부하자. 
  
<div style="page-break-after: always;"></div> 

## 2. 날짜/시각 작업

- 문자열에 대하여 그랬듯이,  
  판다스는 시간 계열 데이터에 대해서도 광범위한 기능을 제공한다. 
- 2.1 항에서 `datetime` 자료형 공부부터 시작하자.

### 2.1 날짜/시각 자료형

#### 2.1.1 파이썬 시계열

- 파이썬 내장 모듈 `datetime`에서 제공하는 `datetime` 및 `timedelta` 자료형에 대해서 공부하자. 
  1. 날짜/시각 자료형
  2. 국가/언어 설정
  3. `datetime` 활용
  4. `timedelta` 활용

1. 날짜/시각 자료형

In [37]:
from datetime import datetime, timedelta        # 모듈에서 특정 클래스를 선별적으로 수입

In [38]:
now = datetime.now()                            # datetime 클래스가 제공하는 now() 메소드로 현재 시각 획득
now

datetime.datetime(2021, 9, 30, 22, 23, 37, 202803)

In [39]:
# strftime()은 날짜/시각을 문자열로 변환하는 메소드
(now, 
 now.strftime('%B'),                            # 달 이름
 now.strftime('%A'))                            # 요일 이름 

(datetime.datetime(2021, 9, 30, 22, 23, 37, 202803), 'September', 'Thursday')

In [40]:
# timedelta()는 시간 간격을 처리하는 클래스
now + timedelta(days=1)                        # 하루 후 확인                      

datetime.datetime(2021, 10, 1, 22, 23, 37, 202803)

2. 국가 및 언어 설정

In [41]:
import locale                                  # 모듈 수입

In [42]:
locale.getlocale()                             # 로케일 확인

('Korean_Korea', '949')

In [43]:
(now, 
 now.strftime('%B'),                           # 달 이름
 now.strftime('%A'))                           # 요일 이름 

(datetime.datetime(2021, 9, 30, 22, 23, 37, 202803), 'September', 'Thursday')

In [44]:
locale.setlocale(locale.LC_ALL, '')            # 모든 범주의 로케일을 기본값으로 설정

'Korean_Korea.949'

In [45]:
# locale.setlocale(locale.LC_ALL, 'Korean_Korea.UTF-8')  # 로케일 변경

In [46]:
locale.getlocale()

('Korean_Korea', '949')

In [47]:
(now, 
 now.strftime('%B'),                           # 달 이름
 now.strftime('%A'))                           # 요일 이름 

(datetime.datetime(2021, 9, 30, 22, 23, 37, 202803), '9월', '목요일')

- `locale`을 초기화하면, 달이나 요일 이름이 한글로 출력된다. 
- 이제 아래와 같이 출력할 수 있다. 

In [48]:
(now, 
 now.strftime('%Y-%m-%d (%A) %H:%M:%S'),            # '%A'로 요일 출력
 now.strftime('%Y년 %B %d일 (%A) %H시 %M분 %S초'))  # '%B'로 월 출력

(datetime.datetime(2021, 9, 30, 22, 23, 37, 202803),
 '2021-09-30 (목요일) 22:23:37',
 '2021년 9월 30일 (목요일) 22시 23분 37초')

3. `datetime` 활용

- 파이썬 `datetime()`으로 `datetime` 자료형을 생성할 수 있다. 

In [49]:
from datetime import datetime                                            # 파이썬 datetime 모듈

date = datetime(year=1919, month=3, day=1, hour=14, minute=0, second=0)  # 삼일절 날짜/시각 생성

- `datetime.strftime()` 메소드에 다양한 출력 형식 지정자를 사용할 수 있다. 

In [50]:
from datetime import datetime                                            # 파이썬 datetime 모듈

date = datetime(year=1919, month=3, day=1, hour=14, minute=0, second=0)  # 삼일절 날짜/시각 생성

print(f"{date.strftime('%Y')}년 "                                        # date.strftime('%Y'), 년
    + f"{date.strftime('%B')} "                                          # date.strftime('%B'), '월'이 포함됨
    + f"{date.strftime('%d')}일 "                                        # date.strftime('%d'), 일
    + f"{date.strftime('%A')}\n"                                         # date.strftime('%A'), '요일'이 포함됨
    + f"{date.strftime('%U')} 번째 주\n"                                 # date.strftime('%U')' 주차, 0부터 시작되는 주차 
    + f"{date.strftime('%j')} 번째 날\n"                                 # date.strftime('%j'), 일차, '선두 '0' 출력
    + f"{date.strftime('%p')} "                                          # date.strftime('%p'), 오전/오후
    + f"{date.strftime('%I')}시 "                                        # date.strftime('%I'), 시, 선두 '0' 출력
    + f"{date.strftime('%M')}분 "                                        # date.strftime('%M'), 분, '00분'으로 출력 
    + f"{date.strftime('%S')}초")                                        # date.strftime('%S'), 초, '00초'로 출력

1919년 3월 01일 토요일
08 번째 주
060 번째 날
오후 02시 00분 00초


- `datetime.strftime()` 메소드 외에도, 다양한 속성과 메소드를 쓸 수 있다. 

In [51]:
요일 = ['월', '화', '수', '목', '금', '토', '일']
date = datetime(year=1919, month=3, day=1, hour=14, minute=0, second=0)  # 삼일절

print(f"{date.year}년 "                                                  # date.year
    + f"{date.month}월 "                                                 # date.month, '월'을 포함해야 함
    + f"{date.day}일 "                                                   # date.day 
    + f"({date.weekday()}) "                                             # date.weekday()
    + f"({요일[date.weekday()]})\n"                                      # 요일[date.weekday()]
    + f"{date.isocalendar()[1]} 번째 주\n"                               # date.isocalendar()[1], year, week, weekday
    + f"{date.timetuple()}\n"                                            # date.timetuple()
    + f"{date.timetuple().tm_yday} 번째 날\n"                            # date.timetuple().tm_yday
    + f"{'오전' if date.timetuple().tm_hour < 12 else '오후'} "          # 오전/오후 
    + f"{date.hour if date.hour <= 12 else date.hour-12}시 "             # date.hour 
    + f"{date.minute}분 "                                                # date.mimute
    + f"{date.second}초")                                                # date.second

1919년 3월 1일 (5) (토)
9 번째 주
time.struct_time(tm_year=1919, tm_mon=3, tm_mday=1, tm_hour=14, tm_min=0, tm_sec=0, tm_wday=5, tm_yday=60, tm_isdst=-1)
60 번째 날
오후 2시 0분 0초


- `strptime()`으로 문자열 표현에서 직접 `datetime`을 생성할 수도 있다.  
- 한글판 파이썬 공식 문서의 [날짜와 시간에 대한 형식](https://docs.python.org/ko/3/library/datetime.html#strftime-and-strptime-format-codes)을 참고하라. 
- `strptime()`은 'string **p**revious time'을 의미하는데, *string* 객체를 datetime 객체로 변환한다.  

In [52]:
date = datetime.strptime("1919-1-1, 14:0:0", "%Y-%m-%d, %H:%M:%S")
date.timetuple()

time.struct_time(tm_year=1919, tm_mon=1, tm_mday=1, tm_hour=14, tm_min=0, tm_sec=0, tm_wday=2, tm_yday=1, tm_isdst=-1)

- `strftime()`으로 datetime 객체로부터 요소를 추출할 수 있다
- `strftime()`은 'string **f**orward time'을 의미하는데, datetime 객체를 *string* 객체로 변환한다.

In [53]:
print(f"  연도: {date.strftime('%Y')}")
print(f"    월: {date.strftime('%m')} -> {int(date.strftime('%m'))}")
print(f"    일: {date.strftime('%d')} -> {int(date.strftime('%d'))}")
print(f"  요일: {date.strftime('%a')}")
print(f"  주차: {date.strftime('%j')}, {int(date.strftime('%j'))}")
print(f"시간대: {date.strftime('%p')}")

  연도: 1919
    월: 01 -> 1
    일: 01 -> 1
  요일: 수
  주차: 001, 1
시간대: 오후


4. `timedelta` 활용

- 파이썬 `timedelta()`로 특정 시각에서 일정 시간이 경과한 시각을 구하는 예제이다:

In [54]:
date

datetime.datetime(1919, 1, 1, 14, 0)

In [55]:
date - timedelta(days=1.5)

datetime.datetime(1918, 12, 31, 2, 0)

In [56]:
date + timedelta(weeks=1)

datetime.datetime(1919, 1, 8, 14, 0)

In [57]:
date - timedelta(hours=1.5)

datetime.datetime(1919, 1, 1, 12, 30)

- 2.1.1 목에서는 파이썬 날짜/시각 자료형을 공부했다.  
  1. 날짜/시각 자료형
  2. 국가/언어 설정
  3. `datetime` 활용
  4. `timedelta` 활용
- 2.1.2 목에서는 넘파이 날짜/시각 자료형을 공부하자. 

#### 2.1.2 넘파이 시계열

- 문자열에서도 그랬듯이,  
  파이썬 `datetime`을 배열에서 처리하는 작업은 어렵고 비효율적이다.  
- 이러한 상황을 고려하여 넘파이에서는  
  - 날짜를 더 효과적으로 처리하는 새로운 `datetime64` 자료형을 제공한다.
  - 넘파이 `datetime64` 자료형은 벡터화를 지원한다. 

- 넘파이 `datetime64` 자료형을 `np.array()` 및 `np.arange()`에서 사용할 수 있다. 

- `np.array()` 메소드로 날짜/시각 배열을 생성할 수 있다. 

In [58]:
dates = np.array(["2020-07-09", "2020-08-10"], dtype="datetime64")
dates                                                           # 개별 날짜를 저장한 넘파이 datetime64[D] 자료형 배열

array(['2020-07-09', '2020-08-10'], dtype='datetime64[D]')

- `np.arange()` 메소드로 날짜/시각 배열을 생성할 수 있다:

In [59]:
dates = np.arange("2020-07", "2020-12", dtype="datetime64")  # [시작, 끝), 월(M) 단위
dates

array(['2020-07', '2020-08', '2020-09', '2020-10', '2020-11'],
      dtype='datetime64[M]')

- 이렇게 만든 배열을 이용해서, 시간에 관한 벡터화 배열 연산을 쉽게 수행할 수 있다.  
  넘파이 `datetime64`의 단위와 형식에 대한 모든 사항에 대한 설명을 [이 문서에서 참고](https://numpy.org/doc/stable/reference/arrays.datetime.html#datetime-units)할 수 있다. 

In [60]:
dates = np.arange("2020-07", "2020-12", dtype="datetime64")  # [시작, 끝), 월(M) 단위
dates

array(['2020-07', '2020-08', '2020-09', '2020-10', '2020-11'],
      dtype='datetime64[M]')

In [61]:
dates + np.timedelta64(2, 'M')                               # 넘파이 timedelta64 객체

array(['2020-09', '2020-10', '2020-11', '2020-12', '2021-01'],
      dtype='datetime64[M]')

- 2.1 항에서 날짜/시간 자료형의 개념을  
  파이썬 및 넘파이 중심으로 공부했다.
- 2.2 항에서 날짜/시간 자료형의 생성 방법을  
  판다스 중심으로 공부하자. 
  
<div style="page-break-after: always;"></div> 

### 2.2 판다스 시계열 생성

- 넘파이가 날짜와 시각을 배열의 세상으로 데려오긴 했지만,  
  데이터 정제에서 일반적으로 필요한 과제를 수행하기에는 상당한 기능적 제약이 존재한다. 
- 부족한 부분은 판다스 라이브러리에게 의지해야 한다.  
  판다스에서는 4 종의 핵심 객체를 제공하는데, 지금부터 이를 공부한다:
  1. (np.datetime64와 유사한) Timestamp 
  2. (np.timedelta64와 유사한) Timedelta 
  3. (날짜/시각의 정규적 범위를 다루는 맞춤형 객체인) Period 
  4. (timedelta와 유사하지만 달력의 규칙을 반영한 맞춤형 객체인) DateOffset 

#### 2.2.1 기본적 생성 방법

시계열과 관련하여 보편적으로 원하는 판다스 작업은 다음과 같다:
1. `pd.Timestamp()`로 `1919-03-01 00:00:00`같은 특정 시점을 생성
2. `pd.Period()`로 `1919년 3월`같은 특정 기간을 생성
3. `pd.date_range()` 및 `pd.period_range()`로 datetime 배열을 생성

1. `pd.Timestamp()`로 특정 시점을 생성하는 방법부터 알아보자. 

In [62]:
print(pd.Timestamp('1919-03-01'))                         # 문자열로부터
print(pd.Timestamp(year=1919, month=3, day=1))            # 날짜 시각의 요소로부터
print(pd.Timestamp(datetime(year=1919, month=3, day=1)))  # datetime 객체로부터
                                                          # 모두 특정 시점

1919-03-01 00:00:00
1919-03-01 00:00:00
1919-03-01 00:00:00


In [63]:
print(pd.Timestamp('1919-03-01 14:3:5'))                                               # 문자열로부터
print(pd.Timestamp(year=1919, month=3, day=1, hour=14, minute=3, second=5))            # 날짜 시각의 요소로부터
print(pd.Timestamp(datetime(year=1919, month=3, day=1, hour=14, minute=3, second=5)))  # datetime 객체로부터
                                                                                       # 모두 특정 시점

1919-03-01 14:03:05
1919-03-01 14:03:05
1919-03-01 14:03:05


2. 직전 예제에서는 `pd.Timestamp()`로 특정 시점을 생성했지만,  
  아래 예제에서는 특정 기간을 생성하기 위하여 `pd.Period()`를 사용한다:

In [64]:
span = pd.Period('1919-03-01')  # 날짜를 기간으로 생성

print(span)
print(span.start_time)          # 기간 시작 시점
print(span.end_time)            # 기간 종료 시점

1919-03-01
1919-03-01 00:00:00
1919-03-01 23:59:59.999999999


In [65]:
시점 = pd.Timestamp('1919-03-01 12:00')
기간 = pd.Period('1919-03-01')

print(f"시점: {시점}")
print(f"기간: {기간}")
print(f"시점이 기간에 포함되는지? {기간.start_time <= 시점 <= 기간.end_time}")

시점: 1919-03-01 12:00:00
기간: 1919-03-01
시점이 기간에 포함되는지? True


3. 때로는 단일 값이 아니라 datetime의 배열을 생성하고 싶다.  
   datetime의 배열로 `DatetimeIndex`/`PeriodIndex`/`TimedeltaIndex` 클래스를 생성한다:

- `pd.date_range()`는 `DatetimeIndex`를 반환한다. 

In [66]:
pd.date_range('1919-03-01 12:00',
              '1919-03-11 12:00',  # 종료 시점도 포함됨 
              freq='D')

DatetimeIndex(['1919-03-01 12:00:00', '1919-03-02 12:00:00',
               '1919-03-03 12:00:00', '1919-03-04 12:00:00',
               '1919-03-05 12:00:00', '1919-03-06 12:00:00',
               '1919-03-07 12:00:00', '1919-03-08 12:00:00',
               '1919-03-09 12:00:00', '1919-03-10 12:00:00',
               '1919-03-11 12:00:00'],
              dtype='datetime64[ns]', freq='D')

- `pd.period_range()`는 `PeriodIndex`를 반환한다. 

In [67]:
pd.period_range('1919-03-01',
                '1919-03-11',      # 종료 기간도 포함됨
                freq='D')

PeriodIndex(['1919-03-01', '1919-03-02', '1919-03-03', '1919-03-04',
             '1919-03-05', '1919-03-06', '1919-03-07', '1919-03-08',
             '1919-03-09', '1919-03-10', '1919-03-11'],
            dtype='period[D]')

- `pd.Timedelta()`는 `TimedeltaIndex`를 반환한다.  
  `pd.Timedelta()`를 써서 시간에 대한 덧셈이나 뺄셈을 벡터화 방식으로 처리할 수 있다: 

In [68]:
pd.date_range('1919-03-01 12:00', '1919-03-11 12:00', freq='D') + pd.Timedelta('1.5 hour')

DatetimeIndex(['1919-03-01 13:30:00', '1919-03-02 13:30:00',
               '1919-03-03 13:30:00', '1919-03-04 13:30:00',
               '1919-03-05 13:30:00', '1919-03-06 13:30:00',
               '1919-03-07 13:30:00', '1919-03-08 13:30:00',
               '1919-03-09 13:30:00', '1919-03-10 13:30:00',
               '1919-03-11 13:30:00'],
              dtype='datetime64[ns]', freq='D')

In [69]:
pd.date_range('1919-03-01 12:00', '1919-03-11 12:00', freq='D') + pd.Timedelta('1.9 hour')

DatetimeIndex(['1919-03-01 13:54:00', '1919-03-02 13:54:00',
               '1919-03-03 13:54:00', '1919-03-04 13:54:00',
               '1919-03-05 13:54:00', '1919-03-06 13:54:00',
               '1919-03-07 13:54:00', '1919-03-08 13:54:00',
               '1919-03-09 13:54:00', '1919-03-10 13:54:00',
               '1919-03-11 13:54:00'],
              dtype='datetime64[ns]', freq='D')

- 마지막으로, 판다스는 날짜/시간 결측치를 `pd.NaT`(Not a Time)로 표시하는데, 이는 `np.NaN`과 유사한 것이다: 

In [70]:
pd.Timestamp(pd.NaT)

NaT

- 2.2.1 목에서 날짜/시간 자료형의 기본적 생성 방법을 공부했다.
    1. 특정 시점을 생성하는 `pd.Timestamp()`
    2. 특정 기간을 생성하는 `pd.Period()`
    3. 시계열 배열을 생성하는 `pd.date_range()` 및 `pd.period_range()` 
- 2.2.2 목에서 기존 자료를 날짜/시간 자료형으로 변환하는 방법을 공부하자. 
  
<div style="page-break-after: always;"></div> 

#### 2.2.2 기존 자료 변환

- 기존 날짜/시각 자료는 보통 문자열로 존재한다. 
- `pd.to_datetime()`으로 문자열을 판다스 날짜/시각으로 변환한다. 
  - 날짜 문자열이 저장된 리스트
  - 날짜 요소가 저장된 리스트를 항목 값으로 가지는 사전

- 일상적으로 날짜는 문자열 리스트로 존재한다.  
  `pd.to_datetime()`을 써서 문자열 리스트를 판다스 `DatetimeIndex`로 변환한다:

In [71]:
string_dates = ['July 9, 2020', 'August 1, 2020', 'August 28, 2020']  # 영문 형식 
pd.to_datetime(string_dates)

DatetimeIndex(['2020-07-09', '2020-08-01', '2020-08-28'], dtype='datetime64[ns]', freq=None)

In [72]:
string_dates = ['2020 9 July', '2020 1 August', '2020 28 August']     # 다른 형식
pd.to_datetime(string_dates)                                

DatetimeIndex(['2020-07-09', '2020-08-01', '2020-08-28'], dtype='datetime64[ns]', freq=None)

In [73]:
string_dates = ['2020-07-09', '2020-08-01', '2020-08-28']             # 또 다른 형식
pd.to_datetime(string_dates)

DatetimeIndex(['2020-07-09', '2020-08-01', '2020-08-28'], dtype='datetime64[ns]', freq=None)

- 아니면 사전을 이용하여 날짜/시각 시리즈를 생성하고,  
  이를 판다스 인덱스로 변환하여 `DatetimeIndex`를 생성한다:

In [74]:
dict_dates = pd.to_datetime({"year": [2020, 2020, 2020],
                             "month": [7, 8, 8],
                             "day": [9, 1, 28]})         
dict_dates                                                             # 단일 열, 시리즈!                                             

0   2020-07-09
1   2020-08-01
2   2020-08-28
dtype: datetime64[ns]

In [75]:
pd.Index(dict_dates)                                                  # 시리즈를 인덱스로 변환 

DatetimeIndex(['2020-07-09', '2020-08-01', '2020-08-28'], dtype='datetime64[ns]', freq=None)

- 더 복잡한 판다스 `datetime64` 자료형에 대해서는, [한글판 파이썬 공식 문서의 자료](https://docs.python.org/ko/3/library/datetime.html#strftime-and-strptime-format-codes)를 참고하라:

- 2.2.2 목에서 기존 자료를 날짜/시간 자료형으로 변환하는 방법을 공부했다.
- 2.2.3 목에서 파일 자료로부터 날짜/시간 자료를 입력하는 방법을 공부하자. 
  
<div style="page-break-after: always;"></div> 

#### 2.2.3 파일 자료 입력

- 자전거 출근 데이터 집합으로부터 날짜/시각 입력을 연습해 보자.  
  - 'cycling_data.csv' 파일에 담긴 데이터는 다음과 같다.
  - "날짜, 시각" 데이터가 단위 데이터라는 의미에서 큰 따옴표 내부에 들어 있다. 
```
Date,Name,Type,Time,Distance,Comments
"10 Sep 2019, 00:13:04",Afternoon Ride,Ride,2084,12.62,Rain
"10 Sep 2019, 13:52:18",Morning Ride,Ride,2531,13.03,rain
...
```

In [76]:
df = pd.read_csv('../data/cycling_data.csv') 
df.head()

,Date,Name,Type,Time,Distance,Comments
0,"10 Sep 2019, 00:13:04",Afternoon Ride,Ride,2084,12.62,Rain
1,"10 Sep 2019, 13:52:18",Morning Ride,Ride,2531,13.03,rain
2,"11 Sep 2019, 00:23:50",Afternoon Ride,Ride,1863,12.52,Wet road but nice weather
3,"11 Sep 2019, 14:06:19",Morning Ride,Ride,2192,12.84,Stopped for photo of sunrise
4,"12 Sep 2019, 00:28:05",Afternoon Ride,Ride,1891,12.48,Tired by the end of the week


- df에서 'Date' 열은 문자열이다. 

In [77]:
print(df['Date'].dtype)

object


In [78]:
df['Date'].head()

0    10 Sep 2019, 00:13:04
1    10 Sep 2019, 13:52:18
2    11 Sep 2019, 00:23:50
3    11 Sep 2019, 14:06:19
4    12 Sep 2019, 00:28:05
Name: Date, dtype: object

- 원한다면, 날짜형으로 변환할 수 있다. 

In [79]:
pd.to_datetime(df['Date']).head()

0   2019-09-10 00:13:04
1   2019-09-10 13:52:18
2   2019-09-11 00:23:50
3   2019-09-11 14:06:19
4   2019-09-12 00:28:05
Name: Date, dtype: datetime64[ns]

- 날짜형으로 변환한 후에는, 특정 요소에 `.dt` 속성을 통하여 접근할 수 있다. 
- 이와 같이 문자열 열을 날짜형 열로 변환하는 판다스 방법에 대해서는 [이 자료](https://towardsdatascience.com/4-tricks-you-should-know-to-parse-date-columns-with-pandas-read-csv-27355bb2ad0e)를 참고하라. 

In [80]:
pd.to_datetime(df['Date']).dt.year

0     2019
1     2019
2     2019
3     2019
4     2019
      ... 
28    2019
29    2019
30    2019
31    2019
32    2019
Name: Date, Length: 33, dtype: int64

- 다른 날짜 형식으로 저장된 데이터도 처리 가능하다. 

```
Date,Name,Type,Time,Distance,Comments
2019-09-10 00:13:04,Afternoon Ride,Ride,2084,12.62,Rain
2019-09-10 13:52:18,Morning Ride,Ride,2531,13.03,rain
...
```

- 이번에는 읽어들이면서 `parse_dates` 옵션을 지정하여, 날짜형으로 변환하자. 

In [81]:
df2 = pd.read_csv('../data/cycling_data_2.csv', 
                  parse_dates=['Date'])
df2.head()

,Date,Name,Type,Time,Distance,Comments
0,2019-09-10 00:13:04,Afternoon Ride,Ride,2084,12.62,Rain
1,2019-09-10 13:52:18,Morning Ride,Ride,2531,13.03,rain
2,2019-09-11 00:23:50,Afternoon Ride,Ride,1863,12.52,Wet road but nice weather
3,2019-09-11 14:06:19,Morning Ride,Ride,2192,12.84,Stopped for photo of sunrise
4,2019-09-12 00:28:05,Afternoon Ride,Ride,1891,12.48,Tired by the end of the week


- 날짜형이므로 `.dt` 속성을 통해서 다양한 날짜/시간 속성에 접근이 가능하다.  
  `df2['Date']`를 날짜와 시각으로 분리하고,  
  새로운 데이터프레임의 분리된 열로 저장한 결과이다. 

In [82]:
pd.DataFrame({'날짜': df2['Date'].dt.date,            
              '시각': df2['Date'].dt.time}).head()

,날짜,시각
0,2019-09-10,00:13:04
1,2019-09-10,13:52:18
2,2019-09-11,00:23:50
3,2019-09-11,14:06:19
4,2019-09-12,00:28:05


- 자전거 통근 원 데이터를 날짜형 인덱스로 지정하여 읽어보자:

In [83]:
df = pd.read_csv('../data/cycling_data.csv', 
                 index_col=0, 
                 parse_dates=True)
df.head(5)  

,Name,Type,Time,Distance,Comments
Date,,,,,
2019-09-10 00:13:04,Afternoon Ride,Ride,2084,12.62,Rain
2019-09-10 13:52:18,Morning Ride,Ride,2531,13.03,rain
2019-09-11 00:23:50,Afternoon Ride,Ride,1863,12.52,Wet road but nice weather
2019-09-11 14:06:19,Morning Ride,Ride,2192,12.84,Stopped for photo of sunrise
2019-09-12 00:28:05,Afternoon Ride,Ride,1891,12.48,Tired by the end of the week


- 이제 인덱스 상황은 다음과 같이 변화하였다.  
  - `df.index.dtype`로 확인하면,  
    `object`가 아니라 `datetime64[ns]`로 처리되었다.  
  - `type(df.index)`로 확인하면,  
    `pandas.core.indexes.base.Index`가 아니라 `pandas.core.indexes.datetimes.DatetimeIndex`로 처리되었다.

In [84]:
print(df.index.dtype)
print(type(df.index))

datetime64[ns]
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


- `df.index`를 `df.index.date`와 `df.index.time`으로 분리하고,  
  새로운 데이터프레임의 분리된 열로 저장할 수도 있다.

In [85]:
pd.DataFrame({'날짜': df.index.date, 
              '시각': df.index.time}).head()

,날짜,시각
0,2019-09-10,00:13:04
1,2019-09-10,13:52:18
2,2019-09-11,00:23:50
3,2019-09-11,14:06:19
4,2019-09-12,00:28:05


- `parse_dates` 인자는 매우 유연하여, 해석하기 난해한 datetime 형식도 지정이 가능하다. 
- `date_parser`, `dayfirst` 등과 같은 관련 인자도 제공되니,  
  [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)을 확인해 보라. 

- 2.2 항에서 날짜/시간 자료형을 생성하는 방법을 공부했다.
  - 기본적 생성 방법
    - pd.Timestamp()로 시점 생성
    - pd.Period()로 기간 생성
    - pd.date_range() 및 pd.period_range()로 datetime 배열 생성
  - 기존 자료 변환
  - 파일 자료 입력
- 2.3 항에서 날짜/시간 자료에 접근하는 인덱싱 방법을 공부하자. 
  
<div style="page-break-after: always;"></div> 

### 2.3 판다스 시계열 인덱싱

- datetime 인덱스 객체도 정규 Index 객체와 유사하게,  
  선택, 슬라이싱, 필터링 등이 가능하다. 

In [86]:
df

,Name,Type,Time,Distance,Comments
Date,,,,,
2019-09-10 00:13:04,Afternoon Ride,Ride,2084,12.62,Rain
2019-09-10 13:52:18,Morning Ride,Ride,2531,13.03,rain
2019-09-11 00:23:50,Afternoon Ride,Ride,1863,12.52,Wet road but nice weather
2019-09-11 14:06:19,Morning Ride,Ride,2192,12.84,Stopped for photo of sunrise
2019-09-12 00:28:05,Afternoon Ride,Ride,1891,12.48,Tired by the end of the week
...,...,...,...,...,...
2019-10-04 01:08:08,Afternoon Ride,Ride,1870,12.63,"Very tired, riding into the wind"
2019-10-09 13:55:40,Morning Ride,Ride,2149,12.70,Really cold! But feeling good
2019-10-10 00:10:31,Afternoon Ride,Ride,1841,12.59,Feeling good after a holiday break!


- 부분적인 스트링 인덱싱을 다음과 같이 수행할 수 있다:

In [87]:
df.loc['2019-09']  # df.loc[행] 또는 df.loc[행, 열] 형식

,Name,Type,Time,Distance,Comments
Date,,,,,
2019-09-10 00:13:04,Afternoon Ride,Ride,2084,12.62,Rain
2019-09-10 13:52:18,Morning Ride,Ride,2531,13.03,rain
2019-09-11 00:23:50,Afternoon Ride,Ride,1863,12.52,Wet road but nice weather
2019-09-11 14:06:19,Morning Ride,Ride,2192,12.84,Stopped for photo of sunrise
2019-09-12 00:28:05,Afternoon Ride,Ride,1891,12.48,Tired by the end of the week
...,...,...,...,...,...
2019-09-25 13:35:41,Morning Ride,Ride,2124,12.65,Stopped for photo of sunrise
2019-09-26 00:13:33,Afternoon Ride,Ride,1860,12.52,raining
2019-09-26 13:42:43,Morning Ride,Ride,2350,12.91,Detour around trucks at Jericho


- 완전 매칭 인덱싱을 다음과 같이 수행할 수 있다:

In [88]:
df.loc['2019-10-10']

,Name,Type,Time,Distance,Comments
Date,,,,,
2019-10-10 00:10:31,Afternoon Ride,Ride,1841,12.59,Feeling good after a holiday break!
2019-10-10 13:47:14,Morning Ride,Ride,2463,12.79,Stopped for photo of sunrise


In [89]:
df.loc['2019-10-10 13:47:14']

,Name,Type,Time,Distance,Comments
Date,,,,,
2019-10-10 13:47:14,Morning Ride,Ride,2463,12.79,Stopped for photo of sunrise


- 슬라이싱을 다음과 같이 수행할 수 있다:

In [90]:
df.sort_index().loc['2019-10-01':'2019-10-13']  
# df.loc['2019-10-01':'2019-10-13']             # 정렬 없는 슬라이싱은 미래 경고를 유발함 

,Name,Type,Time,Distance,Comments
Date,,,,,
2019-10-01 00:15:07,Afternoon Ride,Ride,1732,NaN,Legs feeling strong!
2019-10-01 13:45:55,Morning Ride,Ride,2222,12.82,Beautiful morning! Feeling fit
2019-10-02 00:13:09,Afternoon Ride,Ride,1756,NaN,A little tired today but good weather
2019-10-02 13:46:06,Morning Ride,Ride,2134,13.06,Bit tired today but good weather
2019-10-03 00:45:22,Afternoon Ride,Ride,1724,12.52,Feeling good
2019-10-03 13:47:36,Morning Ride,Ride,2182,12.68,Wet road
2019-10-04 01:08:08,Afternoon Ride,Ride,1870,12.63,"Very tired, riding into the wind"
2019-10-09 13:55:40,Morning Ride,Ride,2149,12.70,Really cold! But feeling good
2019-10-10 00:10:31,Afternoon Ride,Ride,1841,12.59,Feeling good after a holiday break!


- `df.between_time()`를 쓰면 두 시각 사이의 모든 데이터를 검색할 수 있다:

In [91]:
df.between_time('00:00', '01:00')  # 자정 무렵인데, 오후 ...?

,Name,Type,Time,Distance,Comments
Date,,,,,
2019-09-10 00:13:04,Afternoon Ride,Ride,2084,12.62,Rain
2019-09-11 00:23:50,Afternoon Ride,Ride,1863,12.52,Wet road but nice weather
2019-09-12 00:28:05,Afternoon Ride,Ride,1891,12.48,Tired by the end of the week
2019-09-17 00:15:47,Afternoon Ride,Ride,1973,12.45,Legs feeling strong!
2019-09-18 00:15:52,Afternoon Ride,Ride,2101,12.48,Pumped up tires
2019-09-19 00:30:01,Afternoon Ride,Ride,48062,12.48,Feeling good
2019-09-24 00:35:42,Afternoon Ride,Ride,2076,12.47,"Oiled chain, bike feels smooth"
2019-09-25 00:07:21,Afternoon Ride,Ride,1775,12.10,Feeling really tired
2019-09-26 00:13:33,Afternoon Ride,Ride,1860,12.52,raining


- 더 복잡한 필터링을 위해서는, 시계열(timeserie) 분할이 필요한데,  
  이에 대해서는 다음 절에서 공부한다.

- 2.3 항에서 날짜/시간 자료를 인덱싱하는 방법을 공부했다. 
- 2.4 항에서 날짜/시간 자료를 처리하는 방법을 공부하자. 
  
<div style="page-break-after: always;"></div> 

### 2.4 판다스 시계열 처리

#### 2.4.1 요소 추출

- 시간과 날짜에 관한 구성 요소를 분해하여 추출할 수 있다.  
  [구성 요소를 정의하는 많은 속성](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#time-date-components)이 있다. 

In [92]:
df.index.year  # 연도

Int64Index([2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019,
            2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019,
            2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019],
           dtype='int64', name='Date')

In [93]:
df.index.second  # 초

Int64Index([ 4, 18, 50, 19,  5, 48, 47, 34, 53, 52,  1,  9,  5, 41, 42, 24, 21,
            41, 33, 43, 18, 52,  7, 55,  9,  6, 22, 36,  8, 40, 31, 14, 57],
           dtype='int64', name='Date')

In [94]:
df.index.weekday  # 요일(월요일=0, 일요일=6)

Int64Index([1, 1, 2, 2, 3, 0, 1, 1, 2, 2, 3, 3, 4, 0, 1, 1, 2, 2, 3, 3, 4, 0,
            1, 1, 2, 2, 3, 3, 4, 2, 3, 3, 4],
           dtype='int64', name='Date')

In [95]:
요일 = ['월', '화', '수', '목', '금', '토', '일']
요일_리스트 = [요일[_] for _ in df.index.weekday]
print(요일_리스트)

['화', '화', '수', '수', '목', '월', '화', '화', '수', '수', '목', '목', '금', '월', '화', '화', '수', '수', '목', '목', '금', '월', '화', '화', '수', '수', '목', '목', '금', '수', '목', '목', '금']


- 속성뿐만이 아니라 메소드도 사용할 수 있다:

In [96]:
df.index.day_name()                          # 영어 요일

Index(['Tuesday', 'Tuesday', 'Wednesday', 'Wednesday', 'Thursday', 'Monday',
       'Tuesday', 'Tuesday', 'Wednesday', 'Wednesday', 'Thursday', 'Thursday',
       'Friday', 'Monday', 'Tuesday', 'Tuesday', 'Wednesday', 'Wednesday',
       'Thursday', 'Thursday', 'Friday', 'Monday', 'Tuesday', 'Tuesday',
       'Wednesday', 'Wednesday', 'Thursday', 'Thursday', 'Friday', 'Wednesday',
       'Thursday', 'Thursday', 'Friday'],
      dtype='object', name='Date')

In [97]:
df.index.day_name(locale='Korean_Korea.949')  # 한글 요일

Index(['화요일', '화요일', '수요일', '수요일', '목요일', '월요일', '화요일', '화요일', '수요일', '수요일',
       '목요일', '목요일', '금요일', '월요일', '화요일', '화요일', '수요일', '수요일', '목요일', '목요일',
       '금요일', '월요일', '화요일', '화요일', '수요일', '수요일', '목요일', '목요일', '금요일', '수요일',
       '목요일', '목요일', '금요일'],
      dtype='object', name='Date')

In [98]:
df.index.month_name()                      # 월

Index(['September', 'September', 'September', 'September', 'September',
       'September', 'September', 'September', 'September', 'September',
       'September', 'September', 'September', 'September', 'September',
       'September', 'September', 'September', 'September', 'September',
       'September', 'September', 'October', 'October', 'October', 'October',
       'October', 'October', 'October', 'October', 'October', 'October',
       'October'],
      dtype='object', name='Date')

In [99]:
df.index.month_name(locale='Korean_Korea.949')  # 한글 월 

Index(['9월', '9월', '9월', '9월', '9월', '9월', '9월', '9월', '9월', '9월', '9월', '9월',
       '9월', '9월', '9월', '9월', '9월', '9월', '9월', '9월', '9월', '9월', '10월',
       '10월', '10월', '10월', '10월', '10월', '10월', '10월', '10월', '10월', '10월'],
      dtype='object', name='Date')

- `DatetimeIndex` 객체가 아닌  
  데이터프레임의 일반 열과 같은 `Series`에 대해서 작업한다면,  
  `.dt` 속성을 통해서 처리가 가능하다:

In [100]:
s = pd.Series(pd.date_range('2011-12-29', '2011-12-31'))
s

0   2011-12-29
1   2011-12-30
2   2011-12-31
dtype: datetime64[ns]

In [101]:
s.year  # .dt 속성을 통하지 않으면 오류가 발생함

AttributeError: 'Series' object has no attribute 'year'

In [102]:
s.dt.year  # .dt 속성을 통해야 작동함

0    2011
1    2011
2    2011
dtype: int64

- 2.4.1 목에서 날짜/시간 자료에서 요소를 추출하는 방법을 공부했다. 
- 2.4.2 목에서 시차 및 시간대를 처리하는 방법을 공부하자. 
  
<div style="page-break-after: always;"></div> 

#### 2.4.2 시차 및 시간대

- 앞에서 `Timedelta`를 써서 특정 날짜와 시각에 대한  
  덧셈/뺄셈을 수행하는 방법을 살펴 보았다. 
  - `Timedelta`는 절대 시간을 준용하는데,  
    이렇게 하면 시간이 정규적이지 않은 일부 지역의 경우에는 문제가 될 수 있다.   
  - 예를 들어서 캐나다에서는 3월 8일부터 일광 절약 시간제가 시작되는데,  
    이때는 시계를 **한 시간 앞으로** 조정한다.  
  - 이런 '달력 상의 시간' 문제는 절대 시간을 준용하는 `Timedelta`에서는 처리가 어렵다. 

In [103]:
t1 = pd.Timestamp('2020-03-07 12:00:00', tz='Canada/Pacific')  # 3/7 정오를 캐나다 시간대로 지정
t2 = t1 + pd.Timedelta("1 day")                                # (일광 절약 시간제가 시작되는) 하루 뒤를 알고 싶음
print(f"t1: {t1}")
print(f"t2: {t2}")                                             # 시각이 12:00 -> 13:00로 변경됨

t1: 2020-03-07 12:00:00-08:00
t2: 2020-03-08 13:00:00-07:00


- `Timedelta` 대신에 `DateOffset`을 사용할 수 있다:

In [104]:
t3 = t1 + pd.DateOffset(days=1)
print(f"t1: {t1}")
print(f"t3: {t3}")                                             # 시각이 12:00에 머물러 있음

t1: 2020-03-07 12:00:00-08:00
t3: 2020-03-08 12:00:00-07:00


- 앞선 코드의 실행 결과에는 시간대(time zone)가 포함되어 있다.
- 출력에서 이를 제거하려면 `.tz_localize(None)`를 쓴다. 

In [105]:
t1 = pd.Timestamp('2020-03-07 12:00:00', tz='Canada/Pacific')  # tz 지정
t2 = t1 + pd.Timedelta("1 day")                                # 하루 뒤를 알고 싶음
print(f"t1: {t1.tz_localize(None)}")
print(f"t2: {t2.tz_localize(None)}")                           # 시각이 12:00 -> 13:00로 변경됨

t1: 2020-03-07 12:00:00
t2: 2020-03-08 13:00:00


- 앞선 예제에는 시간대(timezone) 정보가 출력에 포함되었다. 
    - 기본적으로 파이썬 datetime 객체는 시간대를 구별하지 않지만,  
      판다스 Timestamp 객체는 시간대를 구별한다.  
    - 앞선 예제와 같이 시간대를 지정하기 위하여  
      `tz` 인수를 지정하거나,  
      `tz_localize()` 메소드를 사용해야 한다:

In [106]:
# 아래 출력에서는 지정된 시간대를 확인하기 위하여 .tz 속성을 출력
print(f"    시간대 지정 없는 경우: {pd.Timestamp('2020-03-07 12:00:00').tz}")
print(f"캐나다 시간대 지정한 경우: {pd.Timestamp('2020-03-07 12:00:00', tz='Canada/Pacific').tz}")
print(f"캐나다 시간대 지정한 경우: {pd.Timestamp('2020-03-07 12:00:00').tz_localize('Canada/Pacific').tz}")
print(f"  한국 시간대 지정한 경우: {pd.Timestamp('2020-03-07 12:00:00').tz_localize('Asia/Seoul').tz}")

    시간대 지정 없는 경우: None
캐나다 시간대 지정한 경우: Canada/Pacific
캐나다 시간대 지정한 경우: Canada/Pacific
  한국 시간대 지정한 경우: Asia/Seoul


- `.tz_convert()` 메소드로 시간대를 변환할 수 있다.  
  아마도 자전거 출퇴근 데이터의 시간이 조금 이상하다는 것을 이미 눈치챘을 것이다.:

In [107]:
df = pd.read_csv('../data/cycling_data.csv', index_col=0, parse_dates=True)
df

,Name,Type,Time,Distance,Comments
Date,,,,,
2019-09-10 00:13:04,Afternoon Ride,Ride,2084,12.62,Rain
2019-09-10 13:52:18,Morning Ride,Ride,2531,13.03,rain
2019-09-11 00:23:50,Afternoon Ride,Ride,1863,12.52,Wet road but nice weather
2019-09-11 14:06:19,Morning Ride,Ride,2192,12.84,Stopped for photo of sunrise
2019-09-12 00:28:05,Afternoon Ride,Ride,1891,12.48,Tired by the end of the week
...,...,...,...,...,...
2019-10-04 01:08:08,Afternoon Ride,Ride,1870,12.63,"Very tired, riding into the wind"
2019-10-09 13:55:40,Morning Ride,Ride,2149,12.70,Really cold! But feeling good
2019-10-10 00:10:31,Afternoon Ride,Ride,1841,12.59,Feeling good after a holiday break!


- 자정 무렵에 출근했을리는 없다.  
  - 이 데이터의 시간대 설정은 오류가 있다. 
  - 이 데이터는 (의도적으로) 캐나다 시간을 호주 시간으로 변환해서 기록한 것이다. 
  - 잘못된 시간대를 수정해 보자:

In [108]:
df.index = df.index.tz_localize("Canada/Pacific")   # 캐나다 시간대로 지정하면,  
df                                                  # 오후와 아침이 시각과 불일치

,Name,Type,Time,Distance,Comments
Date,,,,,
2019-09-10 00:13:04-07:00,Afternoon Ride,Ride,2084,12.62,Rain
2019-09-10 13:52:18-07:00,Morning Ride,Ride,2531,13.03,rain
2019-09-11 00:23:50-07:00,Afternoon Ride,Ride,1863,12.52,Wet road but nice weather
2019-09-11 14:06:19-07:00,Morning Ride,Ride,2192,12.84,Stopped for photo of sunrise
2019-09-12 00:28:05-07:00,Afternoon Ride,Ride,1891,12.48,Tired by the end of the week
...,...,...,...,...,...
2019-10-04 01:08:08-07:00,Afternoon Ride,Ride,1870,12.63,"Very tired, riding into the wind"
2019-10-09 13:55:40-07:00,Morning Ride,Ride,2149,12.70,Really cold! But feeling good
2019-10-10 00:10:31-07:00,Afternoon Ride,Ride,1841,12.59,Feeling good after a holiday break!


In [109]:
df.index = df.index.tz_convert("Australia/Sydney")  # 호주 시간대로 지정하면, 
df                                                  # 오후와 아침이 시각과 일치함

,Name,Type,Time,Distance,Comments
Date,,,,,
2019-09-10 17:13:04+10:00,Afternoon Ride,Ride,2084,12.62,Rain
2019-09-11 06:52:18+10:00,Morning Ride,Ride,2531,13.03,rain
2019-09-11 17:23:50+10:00,Afternoon Ride,Ride,1863,12.52,Wet road but nice weather
2019-09-12 07:06:19+10:00,Morning Ride,Ride,2192,12.84,Stopped for photo of sunrise
2019-09-12 17:28:05+10:00,Afternoon Ride,Ride,1891,12.48,Tired by the end of the week
...,...,...,...,...,...
2019-10-04 18:08:08+10:00,Afternoon Ride,Ride,1870,12.63,"Very tired, riding into the wind"
2019-10-10 07:55:40+11:00,Morning Ride,Ride,2149,12.70,Really cold! But feeling good
2019-10-10 18:10:31+11:00,Afternoon Ride,Ride,1841,12.59,Feeling good after a holiday break!


- 적용해야 하는 시차(offset)를 알고 있다면, 이 경우에는 -7 시간인데,  
  `DateOffset`을 사용해서 처리할 수도 있다:

In [110]:
df = pd.read_csv('../data/cycling_data.csv', index_col=0, parse_dates=True)  # 다시 입력하고
df.index = df.index + pd.DateOffset(hours=-7)                                # 시차 적용
df

,Name,Type,Time,Distance,Comments
Date,,,,,
2019-09-09 17:13:04,Afternoon Ride,Ride,2084,12.62,Rain
2019-09-10 06:52:18,Morning Ride,Ride,2531,13.03,rain
2019-09-10 17:23:50,Afternoon Ride,Ride,1863,12.52,Wet road but nice weather
2019-09-11 07:06:19,Morning Ride,Ride,2192,12.84,Stopped for photo of sunrise
2019-09-11 17:28:05,Afternoon Ride,Ride,1891,12.48,Tired by the end of the week
...,...,...,...,...,...
2019-10-03 18:08:08,Afternoon Ride,Ride,1870,12.63,"Very tired, riding into the wind"
2019-10-09 06:55:40,Morning Ride,Ride,2149,12.70,Really cold! But feeling good
2019-10-09 17:10:31,Afternoon Ride,Ride,1841,12.59,Feeling good after a holiday break!


- 2.4.2 목에서 시차 및 시간대를 처리하는 방법을 공부했다. 
- 2.4.3 목에서 재추출 및 집계를 처리하는 방법을 공부하자. 
  
<div style="page-break-after: always;"></div> 

#### 2.4.3 재추출 및 집계

- 시계열 데이터에서 자주 사용하는 작업으로 재추출(resampling)이 있는데,  
  이는 시계열 데이터의 시간 간격을 넓게 또는 좁게 조정하여  
  다시 집계(aggregating)하려는 의도이다. 
- 예를 들자면, `.resample()` 메소드를 써서  
  일별 데이터를 주별 데이터로 재추출하는 것이다. 
- [재추출 및 집계에 관한 한글 자료](https://rfriend.tistory.com/494)를 참고하라. 
- 아래에서는 자전거 출퇴근 시계열 데이터를  
  1일 주기의 시계열 데이터로 재추출하여 집계해보자:

In [111]:
df.index          # 원 인덱스 확인

DatetimeIndex(['2019-09-09 17:13:04', '2019-09-10 06:52:18',
               '2019-09-10 17:23:50', '2019-09-11 07:06:19',
               '2019-09-11 17:28:05', '2019-09-16 06:57:48',
               '2019-09-16 17:15:47', '2019-09-17 06:43:34',
               '2019-09-18 06:49:53', '2019-09-17 17:15:52',
               '2019-09-18 17:30:01', '2019-09-19 06:52:09',
               '2019-09-19 18:02:05', '2019-09-23 06:50:41',
               '2019-09-23 17:35:42', '2019-09-24 06:41:24',
               '2019-09-24 17:07:21', '2019-09-25 06:35:41',
               '2019-09-25 17:13:33', '2019-09-26 06:42:43',
               '2019-09-26 18:00:18', '2019-09-30 06:53:52',
               '2019-09-30 17:15:07', '2019-10-01 06:45:55',
               '2019-10-01 17:13:09', '2019-10-02 06:46:06',
               '2019-10-02 17:45:22', '2019-10-03 06:47:36',
               '2019-10-03 18:08:08', '2019-10-09 06:55:40',
               '2019-10-09 17:10:31', '2019-10-10 06:47:14',
               '2019-10-

In [112]:
df.resample("1D")  # 하루 주기로 재추출

- `resample` 객체는 지난 장에서 공부했던 `groupby` 객체와 비슷하게 활용할 수 있다.  
  `groupby` 객체에서 했던 것처럼,  
  재추출을 통해 조정된 주기로 재 그룹핑된 시계열 데이터에 대하여  
  집계 함수를 적용한다: 

In [113]:
df[['Time', 'Distance']]       # 원 자료를 다시 확인하면 보통 하루에 2건씩 기록된 자료

,Time,Distance
Date,,
2019-09-09 17:13:04,2084,12.62
2019-09-10 06:52:18,2531,13.03
2019-09-10 17:23:50,1863,12.52
2019-09-11 07:06:19,2192,12.84
2019-09-11 17:28:05,1891,12.48
...,...,...
2019-10-03 18:08:08,1870,12.63
2019-10-09 06:55:40,2149,12.70
2019-10-09 17:10:31,1841,12.59


In [114]:
dfr = df.resample("1D").mean()  # 재추출 및 재집계한 자료에 NaN이 많이 보임       
dfr

,Time,Distance
Date,,
2019-09-09,2084.0,12.620
2019-09-10,2197.0,12.775
2019-09-11,2041.5,12.660
2019-09-12,NaN,NaN
2019-09-13,NaN,NaN
...,...,...
2019-10-06,NaN,NaN
2019-10-07,NaN,NaN
2019-10-08,NaN,NaN


- `NaN`이 꽤 많이 보인다.  
  주말이나 휴일에는 출근하지 않았을 것이다.  
  요일을 추가하여 확인하여 보자.

In [115]:
dfr['Weekday'] = dfr.index.day_name()
dfr.head(15)

,Time,Distance,Weekday
Date,,,
2019-09-09,2084.0,12.620,Monday
2019-09-10,2197.0,12.775,Tuesday
2019-09-11,2041.5,12.660,Wednesday
2019-09-12,NaN,NaN,Thursday
2019-09-13,NaN,NaN,Friday
2019-09-14,NaN,NaN,Saturday
2019-09-15,NaN,NaN,Sunday
2019-09-16,2122.5,12.450,Monday
2019-09-17,2193.0,12.540,Tuesday


- 요일을 우리말로 출력하여 보자. 

In [116]:
days = {0:'월', 1:'화', 2:'수', 3:'목', 4:'금', 5:'토', 6:'일'}

dfr['Weekday'] = dfr.index.day_name()
dfr['요일 번호'] = dfr.index.dayofweek
dfr['요일 이름'] = dfr['요일 번호'].apply(lambda x: days[x])

dfr['한글_요일1'] = dfr.index.day_name(locale='Korean_Korea.949')
dfr['한글_요일2'] = dfr.index.day_name(locale='Korean_Korea.949').str.replace('요일', '')
dfr.head(10)

,Time,Distance,Weekday,요일 번호,요일 이름,한글_요일1,한글_요일2
Date,,,,,,,
2019-09-09,2084.0,12.620,Monday,0,월,월요일,월
2019-09-10,2197.0,12.775,Tuesday,1,화,화요일,화
2019-09-11,2041.5,12.660,Wednesday,2,수,수요일,수
2019-09-12,NaN,NaN,Thursday,3,목,목요일,목
2019-09-13,NaN,NaN,Friday,4,금,금요일,금
2019-09-14,NaN,NaN,Saturday,5,토,토요일,토
2019-09-15,NaN,NaN,Sunday,6,일,일요일,일
2019-09-16,2122.5,12.450,Monday,0,월,월요일,월
2019-09-17,2193.0,12.540,Tuesday,1,화,화요일,화


- 판다스는 업무 시간(business time) 개념을 지원하며,  
  앞서 본 모든 시계열 함수에서 업무 시간 관련 형식 코드를 지원한다. 
- 주말을 건너 띄는 업무 요일 개념을 살펴보자: 

In [117]:
금요일 = pd.Timestamp("2018-01-05")
토요일 = 금요일 + pd.Timedelta("1 day")    # 일반적 1일 더하기
월요일 = 금요일 + pd.offsets.BDay()        # 업무일 1일 더하기
토요일.day_name(locale='Korean_Korea.949'), 월요일.day_name(locale='Korean_Korea.949')

('토요일', '월요일')

In [118]:
dfr = df.resample("1B").mean()             # "B"는 업무일(business day)을 의미
dfr['요일(평일)'] = dfr.index.day_name(    # 평일만 집계됨
    locale='Korean_Korea.949')      
dfr.head(20)                               # 휴일은 배제됨   

,Time,Distance,요일(평일)
Date,,,
2019-09-09,2084.0,12.620,월요일
2019-09-10,2197.0,12.775,화요일
2019-09-11,2041.5,12.660,수요일
2019-09-12,NaN,NaN,목요일
2019-09-13,NaN,NaN,금요일
2019-09-16,2122.5,12.450,월요일
2019-09-17,2193.0,12.540,화요일
2019-09-18,25482.5,13.525,수요일
2019-09-19,2525.5,12.700,목요일


- 지금까지 날짜/시간 처리 방법을 공부했다.  
  - 날짜/시간 요소 추출
  - 시차 및 시간대 
  - 재추출 및 집계
- 이제부터 계층적 인덱싱을 공부하자. 
  
<div style="page-break-after: always;"></div> 